In [1]:
#Import relevant modules
import time
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By

In [2]:
driver = webdriver.Chrome('/Users/Hei/Applications/chromedriver')

In [3]:
suffix_raceno='2' #hardcode for test, default =1
URL_coming_race = 'https://bet.hkjc.com/racing/pages/odds_wp.aspx?lang=en&raceno='+suffix_raceno
URL_coming_race

'https://bet.hkjc.com/racing/pages/odds_wp.aspx?lang=en&raceno=2'

In [4]:
# Initialize soup for getting number of race
driver.get(URL_coming_race)
time.sleep(5)
subhtml = driver.page_source
soup = BeautifulSoup(subhtml, 'html.parser')

# Initialize dataframe
df_race = pd.DataFrame(columns=['season','horse','jockey','trainer','class','track','location',\
                                'actual_weight','draw','distance','win_odds','race','date'])

# Get location
location=soup.find('div',{'class':'mtgInfoDV'}).text.split(',')[2].strip()

In [5]:
def get_no_of_race():
    racebg = soup.find(class_="racebg").contents
    count_race=0
    for i in range(len(racebg)):
        if racebg[i] !='\n' and len(racebg[i].attrs)==4 and 'id' in racebg[i].attrs.keys():
            count_race+=1
    return count_race

def get_race_info():
    race_info=soup.find('span',{'class':'content'}).text.split(',')
    date=race_info[1].strip()
    race_class=race_info[3].strip()
    turf=race_info[4].strip()
    course=race_info[5].strip()
    track=turf+' - '+course
    distance=race_info[6].strip()[:-1]
    return date,race_class,track,distance

def write_table():
    date,race_class,track,distance=get_race_info()
    dict_table={'season':[],'horse':[],'jockey':[],'trainer':[],'class':[],'track':[],'location':[],\
                'actual_weight':[],'draw':[],'distance':[],'win_odds':[],'race':[],'date':[]}
    table = soup.find_all(id='horseTable')[0].tbody.find_all('td')[:-6]
    for i in range(len(table)):
        if i %10==0:
            dict_table['season'].append('2020/2021')
            dict_table['class'].append(race_class)
            dict_table['track'].append(track)
            dict_table['location'].append(location)
            dict_table['distance'].append(distance)
            dict_table['date'].append(date)
            dict_table['race'].append(table[i].text)
        elif i%10==2:
            dict_table['horse'].append(table[i].text)
        elif i%10==3:
            dict_table['draw'].append(table[i].text)
        elif i%10==4:
            dict_table['actual_weight'].append(table[i].text)
        elif i%10==5:
            dict_table['jockey'].append(table[i].text)
        elif i%10==6:
            dict_table['trainer'].append(table[i].text)
        elif i%10==7:
            dict_table['win_odds'].append(table[i].text)
    df=pd.DataFrame(dict_table)
    return df

In [6]:
for i in range(get_no_of_race()):
    # get soup for each race
    URL_coming_race = 'https://bet.hkjc.com/racing/pages/odds_wp.aspx?lang=en&raceno='+str(i+1)
    driver.get(URL_coming_race)
    time.sleep(5)
    subhtml = driver.page_source
    soup = BeautifulSoup(subhtml, 'html.parser')
    
    #get horse inforamtion for each race
    table = soup.find_all(id='horseTable')[0].tbody.find_all('td')[:-6]
    df_race=pd.concat([df_race,write_table()])
df_race

,season,horse,jockey,trainer,class,track,location,actual_weight,draw,distance,win_odds,race,date
0,2020/2021,TINKER BELLE,N Callan,D E Ferraris,Class 5,"TURF - ""C+3"" Course",Happy Valley,133,5,1000,49,1,28/10/2020
1,2020/2021,EVERBRAVE,C Schofield,K L Man,Class 5,"TURF - ""C+3"" Course",Happy Valley,132,9,1000,18,2,28/10/2020
2,2020/2021,SPEEDY OPTIMIST,Z Purton,C H Yip,Class 5,"TURF - ""C+3"" Course",Happy Valley,131,11,1000,11,3,28/10/2020
3,2020/2021,HAPPY WINNER,C Y Ho,C W Chang,Class 5,"TURF - ""C+3"" Course",Happy Valley,127,10,1000,35,4,28/10/2020
4,2020/2021,STRATHCLYDE,C Wong,A T Millard,Class 5,"TURF - ""C+3"" Course",Happy Valley,127,12,1000,23,5,28/10/2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7,2020/2021,ENRICHMENT,C Schofield,J Size,Class 2,"TURF - ""C+3"" Course",Happy Valley,120,4,1800,14,8,28/10/2020
8,2020/2021,AMAZING BEATS,M Chadwick,P O'Sullivan,Class 2,"TURF - ""C+3"" Course",Happy Valley,119,9,1800,19,9,28/10/2020
9,2020/2021,PACKING WALTHAM,K Teetan,A S Cruz,Class 2,"TURF - ""C+3"" Course",Happy Valley,119,3,1800,10,10,28/10/2020
10,2020/2021,TIANCHI MONSTER,M F Poon,W Y So,Class 2,"TURF - ""C+3"" Course",Happy Valley,119,6,1800,8.0,11,28/10/2020


In [7]:
webdriver.quit()
df_race.to_csv('../data/coming_race.csv',index=False)

AttributeError: module 'selenium.webdriver' has no attribute 'quit'

In [ ]:
user_input={'season':'2020/2021','horse':horse.upper(),'jockey':jockey,'trainer':trainer,'class':horse_class,'track':track,'location':location,\
                'actual_weight':actual_weight,'distance':distance,'draw':draw,'prize':prize,'next_race_day':next_race_day}

In [ ]:
cat_vars=['season','last_place','horse','jockey','trainer','class','track','location']
cont_vars=['actual_weight','last_actual_weight','draw','last_lbw','last_running_position',\
           'prize','distance_m','rest_day','pct_1st_last_j','pct_2nd_last_j','pct_3rd_last_j',\
           'pct_4th_last_j','pct_1st_last_t','pct_2nd_last_t','pct_3rd_last_t','pct_4th_last_t']
batch_pred_cols=cat_vars + cont_vars + ['race','date']